<a href="https://colab.research.google.com/github/gcostagithub/DevOpsLab-HelloWorld/blob/master/C%C3%B3pia_de_C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools":[{"google_search":{}}]}

)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico,data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações no Google",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias da lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo  lançamentos relevantes, com base na quantidade e entusiamo das notícias sobre el.
        Se um tema tiver poucas notícias ou reações estusiamadas, é possível que ele não seja tão relavante assim
        e pode ser substituído por outro que tenhas mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lanaçamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher os temas mais relevantes entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos então criar o poste sobre novidade4s em {topico} ")
  lancamentos_buscados = agente_buscador(topico, data_de_hoje)

  print("\n-- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("-----------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n-- Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("-----------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n-- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n-- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("-----------------------------------------------------")







🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA
Maravilha! Vamos então criar o poste sobre novidade4s em agentes de IA 

-- Resultado do Agente 1 (Buscador) ---



> Tudo bem, estou pronto para pesquisar os lançamentos mais relevantes de agentes de IA no último mês. Aqui estão minhas buscas iniciais:
> 
> 
> Com base nas minhas pesquisas, aqui estão alguns dos lançamentos mais relevantes de agentes de IA no último mês:
> 
> 1.  **IBM watsonx Orchestrate:** A IBM anunciou atualizações para o watsonx Orchestrate, oferecendo soluções para integrar, customizar e implantar agentes de IA. Inclui agentes pré-construídos para RH, vendas e compras, com integração a mais de 80 aplicações empresariais. O Agent Catalog simplifica a implantação.
> 2.  **Microsoft Copilot Studio:** A Microsoft apresentou o Copilot Studio, uma solução para criar modelos capazes de automatizar tarefas com o suporte de IA.
> 3.  **Google Gemini 2.0:** O Google avançou com seu modelo de IA Gemini 2.0, que realiza buscas mais detalhadas e complexas sobre temas específicos.
> 4.  **OpenAI GPT-4.1 e GPT-4.1 Mini:** A OpenAI lançou os modelos GPT-4.1 e GPT-4.1 Mini para usuários do ChatGPT. O GPT-4.1 oferece desempenho mais rápido e melhorias em programação e atendimento a instruções.
> 5.  **Yellowfin 9.15:** A plataforma de business intelligence Yellowfin apresentou a versão 9.15, que incorpora a funcionalidade de Natural Query Language habilitada por IA, permitindo consultas por meio de linguagem natural.
> 
> Além destes, destaco também:
> 
> *   **Tendências de Agentes de IA:** Agentes de IA estão se tornando uma tendência crucial nas empresas em 2025, com 80% das organizações planejando avanços significativos na adoção de tecnologias de IA até o final de 2026. A expectativa é que os agentes operem de forma autônoma, antecipando cenários e interagindo de maneira personalizada.
> *   **Ferramentas para criação de agentes de IA:** Plataformas como Dify, CrewAI, Synthflow e LangChain estão se destacando para a criação de agentes de IA.
> *   **Mercado de Agentes de IA:** O mercado global de agentes de IA deverá atingir US$ 52,62 bilhões até 2030, impulsionado pela necessidade de automação eficiente e decisões rápidas.
> 
> 


-----------------------------------------------------

-- Resultado do Agente 2 (Planejador) ---



> Okay, com base nesses lançamentos e tendências de agentes de IA, vou elaborar um plano de conteúdo para um post de rede social.
> 
> 
> Com base na minha pesquisa, o tema mais relevante para um post de mídia social sobre agentes de IA é:
> 
> **Tema:** **Agentes de IA: A Transformação Empresarial em 2025**
> 
> **Pontos Relevantes:**
> 
> *   **Crescimento Exponencial do Mercado:** O mercado global de agentes de IA deverá atingir US$ 52,62 bilhões até 2030.
> *   **Adoção em Massa:** 80% das organizações planejam adotar tecnologias de IA em larga escala até o final de 2026.
> *   **Autonomia e Personalização:** Agentes de IA operam de forma autônoma, antecipando cenários e interagindo de maneira personalizada.
> *   **Aplicações Diversificadas:** Os agentes de IA estão sendo implementados em diversos setores, incluindo RH, vendas, compras, saúde, finanças e engenharia.
> *   **Ferramentas de Criação:** Plataformas como Dify, CrewAI, Synthflow e LangChain estão impulsionando a criação de agentes de IA.
> 
> **Plano de Conteúdo para o Post:**
> 
> 1.  **Introdução:**
>     *   Comece com uma frase chamativa sobre a ascensão dos agentes de IA e seu impacto no mundo dos negócios.
>     *   Apresente o conceito de agentes de IA como "funcionários digitais" que podem automatizar tarefas e melhorar a eficiência.
> 
> 2.  **Estatísticas Impactantes:**
>     *   Destaque o crescimento projetado do mercado de agentes de IA, mencionando o valor de US$ 52,62 bilhões até 2030.
>     *   Cite a porcentagem de empresas que planejam adotar agentes de IA em larga escala (80% até 2026).
>     *   Mencione o ROI de US$ 3,50 para cada US$ 1 investido em IA.
> 
> 3.  **Benefícios e Aplicações:**
>     *   Explore os benefícios dos agentes de IA, como aumento da produtividade, redução de custos e melhor tomada de decisões.
>     *   Apresente exemplos de aplicações em diferentes setores:
>         *   **RH:** Agentes para recrutamento e onboarding.
>         *   **Vendas:** Agentes para qualificação de leads e geração de conteúdo.
>         *   **Saúde:** Agentes para análise de dados e suporte a decisões clínicas.
>         *   **Engenharia:** Agentes para automação de processos e atendimento ao cliente.
>         *   **Financeiro:** Análise de dados e reconhecimento de padrões para decisões em tempo útil.
>     *   Destaque a capacidade dos agentes de IA de operar de forma autônoma e interagir de maneira personalizada.
> 
> 4.  **Ferramentas e Plataformas:**
>     *   Apresente algumas das principais plataformas para criação de agentes de IA:
>         *   **Dify:** Plataforma open-source para desenvolvimento de aplicações de IA.
>         *   **CrewAI:** Framework para construir sistemas multi-agentes.
>         *   **Synthflow:** Plataforma para criar agentes de voz de IA para call centers.
>         *   **LangChain:** Framework para criar aplicações de processamento de linguagem natural.
>     *   Mencione que essas ferramentas permitem que empresas criem agentes de IA personalizados para atender às suas necessidades específicas.
> 
> 5.  **Exemplos de Sucesso:**
>     *   IBM Watsonx Orchestrate otimiza workflows complexos.
>     *   Microsoft Copilot Studio auxilia na extração de informações de PDFs, onboarding de clientes e suporte ao cliente.
>     *   Yellowfin 9.15 permite consultas em linguagem natural (NLQ).
> 
> 6.  **O Futuro dos Agentes de IA:**
>     *   Enfatize que os agentes de IA são uma tendência crescente e que sua adoção continuará a aumentar nos próximos anos.
>     *   Destaque a importância de se preparar para essa transformação e explorar as oportunidades que os agentes de IA oferecem.
> 
> 7.  **Chamada para Ação:**
>     *   Incentive os seguidores a explorar as ferramentas e plataformas mencionadas.
>     *   Faça uma pergunta para estimular o engajamento, como: "Como você imagina que os agentes de IA podem transformar o seu negócio?".
> 
> **Hashtags:**
> 
> \#InteligenciaArtificial \#IA \#AgentesDeIA \#Automacao \#Inovacao \#TransformacaoDigital \#FuturoDoTrabalho \#Dify \#CrewAI \#Synthflow \#LangChain \#Watsonx \#CopilotStudio \#Yellowfin
> 
> Este plano de conteúdo fornecerá um post informativo e envolvente sobre a crescente importância dos agentes de IA no cenário empresarial atual.
> 


-----------------------------------------------------

-- Resultado do Agente 3 (Redator) ---



> 🚀 Prepare-se para o futuro: os agentes de IA estão chegando para revolucionar o mundo dos negócios! 🤖
> 
> Imagine ter "funcionários digitais" que automatizam tarefas, aumentam a eficiência e impulsionam a inovação. Bem-vindo à era dos agentes de IA!
> 
> 📈 O mercado global de agentes de IA está previsto para atingir **US$ 52,62 bilhões até 2030**! E não para por aí: **80% das empresas** planejam adotar essas tecnologias em larga escala até 2026. 🤯
> 
> 💼 **Onde eles atuam?** RH, vendas, saúde, finanças, engenharia... Os agentes de IA estão transformando diversos setores, impulsionando a produtividade, reduzindo custos e otimizando a tomada de decisões. Eles operam de forma autônoma, antecipando cenários e interagindo de maneira personalizada.
> 
> 🛠️ **Como criar o seu?** Plataformas como Dify, CrewAI, Synthflow e LangChain estão facilitando a criação de agentes de IA personalizados para as necessidades de cada negócio.
> 
> ✨ **Exemplos de sucesso:** IBM Watsonx Orchestrate otimizando workflows complexos, Microsoft Copilot Studio turbinando o onboarding de clientes e Yellowfin 9.15 permitindo consultas em linguagem natural.
> 
> 🤔 E você, como imagina que os agentes de IA podem transformar o seu negócio? Compartilhe suas ideias nos comentários! 👇
> 
> \#InteligenciaArtificial \#IA \#AgentesDeIA \#TransformacaoDigital
> 



-- Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais atraente para o público jovem do Instagram com algumas mudanças:
> 
> *   **Tom:** O tom já está bom, mas podemos adicionar um pouco mais de entusiasmo e usar uma linguagem mais coloquial para atrair a atenção.
> *   **Emojis:** Podemos adicionar mais emojis para tornar o post mais visualmente atraente e expressivo.
> *   **Chamada para ação:** A chamada para ação no final é boa, mas podemos torná-la mais específica e envolvente.
> 
> **Sugestões de melhoria:**
> 
> 🚀 **Se liga no futuro: os agentes de IA tão chegando pra botar pra quebrar no mundo dos negócios!** 🤖
> 
> Imagina ter uns "parças digitais" que fazem tudo no automático, turbinam a produtividade e ainda trazem ideias inovadoras? ✨ Cola com a gente na era dos agentes de IA!
> 
> 📈 A parada é séria: o mercado global de agentes de IA tá na previsão de chegar a **52,62 bilhões de dólares até 2030**! 🤑 E não para por aí: **80% das empresas** tão de olho pra внедрить essas tecnologias em larga escala até 2026. 🤯
> 
> 💼 **Onde eles tão mandando?** RH, vendas, saúde, finanças, engenharia... Os agentes de IA tão transformando tudo, aumentando a produtividade, diminuindo os gastos e dando um help na hora de tomar decisões. 🎯 Eles trabalham sozinhos, prevendo o que vai acontecer e conversando com cada um de um jeito especial.
> 
> 🛠️ **Quer criar o seu?** Relaxa! Plataformas como Dify, CrewAI, Synthflow e LangChain tão aí pra te ajudar a criar agentes de IA personalizados pra sua empresa. 😉
> 
> ✨ **Exemplos de sucesso:** IBM Watsonx Orchestrate organizando os processos, Microsoft Copilot Studio dando um up no onboarding dos clientes e Yellowfin 9.15 permitindo que você faça perguntas como se estivesse conversando com um amigo.
> 
> 🤔 E aí, como você acha que os agentes de IA podem dar um upgrade no seu negócio? Deixa sua ideia aqui nos comentários! 👇
> 
> \#InteligenciaArtificial \#IA \#AgentesDeIA \#TransformacaoDigital
> 


-----------------------------------------------------
